In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import cv2
import torchvision.transforms.functional as cvF

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
img = cv2.imread("img2.png", cv2.IMREAD_GRAYSCALE)

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256, 256)),
])

image = transform(img).to(device).unsqueeze(0)
image.shape

# Features
Examples of Using Features
1. Matching - Extract features from two different images to compare them rather than pixel comparison
2. Structure from Motion - Use features from many images to construct something that has been identified from those images from different viewpoints

## Interest Points
Points in an image that can be detected and are relevant for higher level processing
Often used in applications like image stabilization and structure from motion
Applications
    - Key-Point Matching
        1. Find a set of distinctive key-points
        2. Define a region around each key-point
        3. Extract and normalize the region content
        4. Compute a local descriptor from the normalized region
        5. Match local descriptors
### Possible Approaches
Corner Detection
- Recognize corners by looking at a small region.
    - "Flat" if no change in intensity in all directions
    - "Edge" if no change in intensity along edge direction
    - "Corner" if significant change in intensity in all directions

# Harris Corner Detection
$$E(u, v)=\sum_{x,y}w(x, y)[I(x+u, y+v)-I(x, y)]^2$$
1. Compute $M$ matrix for each window to recover a *cornerness* score $C$.
2. Threshold to find pixels which give large corner response
3. Find the local maximima pixels (non-maximum suppresion)

Steps:
1. Compute image gradients
2. Compute $M$ components as squares of derivatives
3. Gaussian filter with width $s$: $g(I_x^2), g(I_y^2), g(I_x\circ I_y)$
4. Compute cornerness $C = det(M)-\alpha trace(M)^2=g(I_x^2)\circ g(I_y^2)-g(I_x\circ I_y)^2-\alpha[g(I_x^2)+g(I_y^2)]^2$
5. Threshold on $C$ to filter for high cornerness
6. Non-maximal suppression to pick peak corners

In [ ]:
def compute_gradients(image: torch.Tensor):
    dx_kernel = torch.tensor(
        [[1, 0, -1], [2, 0, -2], [1, 0, -1]], 
        device=device, dtype=torch.float
    ).view(1, 1, 3, 3)

    dy_kernel = torch.tensor(
        [[1, 2, 1], [0, 0, 0], [-1, -2, -1]],
        device=device, dtype=torch.float
    ).view(1, 1, 3, 3)

    grad_x = F.conv2d(image, dx_kernel, padding=1)
    grad_y = F.conv2d(image, dy_kernel, padding=1)

    return grad_x, grad_y

def non_maximum_suppression(C, corners, radius):
    dilated = F.max_pool2d(C, kernel_size=(radius*2+1, radius*2+1), stride=1, padding=radius)
    local_max = (C == dilated.squeeze())
    return corners & local_max

In [ ]:
alpha = 0.04
threshold = 0.01

# Compute image gradients
grad_x, grad_y = compute_gradients(image)

# Compute products of gradients
Ixx = grad_x ** 2
Iyy = grad_y ** 2
Ixy = grad_x * grad_y

# Smooth products of gradients with gaussian filter
Ixx = cvF.gaussian_blur(Ixx, 5, 1.5)
Iyy = cvF.gaussian_blur(Iyy, 5, 1.5)
Ixy = cvF.gaussian_blur(Ixy, 5, 1.5)

# Compute cornerness
det_M = Ixx * Iyy - Ixy ** 2
trace_M = Ixx + Iyy
C = det_M - alpha * (trace_M ** 2)

# Threshold on C to filter for high cornerness
corners = C > threshold * C.max()

corners = non_maximum_suppression(C, corners, 3)

# Mark corners
corner_image = image.cpu().numpy()
corner_image[corners.cpu()] = 255

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(img, cmap='gray')
plt.axis("off")

plt.subplot(1, 2, 2)
plt.title("Harris Corner Detection")
plt.imshow(corner_image.squeeze(), cmap='gray')
plt.axis("off")

plt.show()

# Histograms of Oriented Gradients
1. Extract a square window (block) of some size
2. Divide block into a square grid of sub-blocks (cells)
3. Compute orientation histogram of each cell
4. Concatenate the histograms together
5. Normalize $v$ (the concatenation of the histograms)
    - Option 1: Divide $v$ by its Euclidean norm
    - Option 2: Divide $v$ by its $L_1$ norm (sum of all absolute values of $v$)
    - Option 3:
        - Divide $v$ by its Euclidean norm
        - Clip any value over 0.2
        - Divide the resulting $v$ by its Euclidean norm

# Scale Invariant Feature Transform (SIFT)
1. Scale-Space Extrema Detection - Search over multiple scales and image locations
2. Key-Point Localization - Fit a model to determine location and scale, then select key-points based on a measure of stability
3. Orientation Assignment - Compute best orientations for each key-point region
4. Key-Point Description - Use local image gradients at selected scale and rotation to describe each key-point region

Automatic Scale Selection
- Function responses for increasing scale (map different scales to the function results and the highest function result has the best scale)
- Function can be 2nd derivative of Gaussian (Laplacian of Gaussian)